In [68]:
import funcs_utility
import random 
import os
from datetime import datetime

import funcs_supabase
import actions
import file_management
import video_processing


from urllib.parse import urlparse, unquote
import requests


from moviepy.editor import VideoFileClip, concatenate_videoclips, ImageClip  
import moviepy.editor as mpy
from moviepy.video.fx.all import crop
from moviepy.editor import AudioFileClip
from moviepy.editor import VideoFileClip

import pexels_2



In [70]:
class MovieProject:
    def __init__(self, project_id):
        self.project_id = project_id

        self.project_data = funcs_supabase.select_data("video_creator_projects", "id", self.project_id)[0]
        self.scenes_data = actions.get_active_scenes(self.project_id)
        
        
        self.video_clips_list = []
        self.speech_clip = None  # Class variable for storing the speech audio clip

        
        
        
        now = datetime.now()
        formatted_timestamp = now.strftime("%Y%m%d%H%M%S")
        self.project_download_folder = f"composed_videos/{self.project_id}_{formatted_timestamp}"
        # Creating the directory if it doesn't exist
        if not os.path.exists(self.project_download_folder):
            os.makedirs(self.project_download_folder)




    def update_project_data(self, key, value):
        self.project_data[key] = value

    def get_project_data(self, key):
        return self.project_data.get(key, None)


    def add_scene(self, scene_data):
        self.scenes_data.append(scene_data)

    def get_all_scenes(self):
        return self.scenes_data

    def get_scene(self, scene_id):
        for scene in self.scenes_data:
            if scene['id'] == scene_id:
                return scene
        return None

    def update_scene(self, scene_id, key, value):
        scene = self.get_scene(scene_id)
        if scene is not None:
            scene[key] = value
        else:
            print(f"No scene found with ID {scene_id}")

    def get_scene_data(self, scene_id, key):
        scene = self.get_scene(scene_id)
        if scene is not None:
            return scene.get(key, None)
        else:
            print(f"No scene found with ID {scene_id}")
            return None


    def download_all_files_for_project(self):
            filenames = []
            for scene in self.scenes_data:
                scene_id = scene['id']
                media_id = scene.get('final_clip')

                if media_id:
                    # Fetch media information from a database
                    media_info = funcs_supabase.select_data("media", "id", media_id)[0]
                    url = media_info.get('url')
                    media_type = media_info.get('type')
                    url_remote = media_info.get('url_remote')

                    # Use remote URL if the primary URL is missing
                    if not url and url_remote:
                        url = url_remote

                    # Handle missing URL scenarios
                    if not url:
                        print(f"No URL found for media ID {media_id}, skipping...")
                        continue
                
                else:
                    # Default action if media_id is not available
                    url = pexels_2.random_image()
                    media_type = 'image'  # Default to image type

                # Getting the file extension 
                url_extension = file_management.get_url_file_extension(url)
                if not url_extension:
                    url_extension = 'png' if media_type == 'image' else 'mp4'

                # Creating the filename
                filename = f"{self.project_download_folder}/{media_id}.{url_extension}" if media_id else f"{self.project_download_folder}/{scene_id}_random.{url_extension}"
                
                # Downloading the file
                file_management.download_file_from_url(url, filename)
                print(f"Downloaded: {filename}")

                # Updating the scene information with the downloaded file name
                self.update_scene(scene_id, 'downloaded_filename', filename)
                filenames.append(filename)
            return filenames



    def get_clips_for_project(self):
        self.video_clips_list = []  # Reset the list each time this method is called
        for scene in self.scenes_data:
            scene_id = scene['id']
            timings = scene['timings']
            length = timings['length']
            downloaded_filename = scene['downloaded_filename']

            if downloaded_filename.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
                clip = VideoFileClip(downloaded_filename)
            elif downloaded_filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                clip = ImageClip(downloaded_filename, duration=length)
            else:
                print(f"Unsupported format: {downloaded_filename}")
                continue  # Skip this scene if the format is unsupported

            # Assuming trim_clip and crop_clip are predefined functions
            trimmed_clip = video_processing.trim_clip(clip, start_trim=0, end_trim=length)

            height = self.get_project_data('height') or 1080
            width = self.get_project_data('width') or 1920  # Adjusted to a more common width for videos

            resized_clip = video_processing.crop_clip(trimmed_clip, width=900, height=900)

            self.video_clips_list.append(resized_clip)

        return self.video_clips_list
    


    def download_and_create_speech_clip(self):
        speech_audio = self.get_project_data('speech_audio')
        if not speech_audio:
            print("No speech audio URL found.")
            return

        # Assuming download_file_from_url is a predefined function
        speech_filepath = f"{self.project_download_folder}/speech.mp3"
        file_management.download_file_from_url(speech_audio, speech_filepath)

        # Assuming AudioFileClip is a predefined class/method
        self.speech_clip = AudioFileClip(speech_filepath)
        print(f"Speech clip duration: {self.speech_clip.duration}")
    
    
    


    def __str__(self):
        return f"Project ID: {self.project_id}\nProject Data: {self.project_data}\nScenes: {self.scenes_data}"


In [71]:
project_id = 24

import scene_timings
project = MovieProject(project_id)

scenes = project.get_all_scenes()
for scene in scenes :
    if scene.get("timings") is None:
        scene_timings.process(project_id)

        project = MovieProject(project_id)




# filenames = download_all_files_for_project()
# processed_clips = get_clips_for_project()
# make_final_vid(processed_clips)



2023-11-02 20:22:28,688:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/video_creator_projects?select=%2A&id=eq.24 "HTTP/1.1 200 OK"
2023-11-02 20:22:28,798:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/scenes?select=%2A&project_id=eq.24&status=eq.Active "HTTP/1.1 200 OK"


In [72]:
project = MovieProject(project_id)
# Assuming other necessary methods are called to set up the project
project.download_and_create_speech_clip()
speech_clip = project.speech_clip

# Now you can use speech_clip as required
if speech_clip is not None:
    print(f"Speech clip duration: {speech_clip.duration}")
else:
    print("Speech clip is not available.")

project.download_all_files_for_project()
project.get_clips_for_project()


2023-11-02 20:22:32,752:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/video_creator_projects?select=%2A&id=eq.24 "HTTP/1.1 200 OK"
2023-11-02 20:22:32,867:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/scenes?select=%2A&project_id=eq.24&status=eq.Active "HTTP/1.1 200 OK"


Speech clip duration: 44.56
Speech clip duration: 44.56
Random image URL: https://images.pexels.com/photos/18814109/pexels-photo-18814109.jpeg
Downloaded: composed_videos/24_20231102202232/218_random.jpeg
Random image URL: https://images.pexels.com/photos/18802849/pexels-photo-18802849.jpeg
Downloaded: composed_videos/24_20231102202232/219_random.jpeg
Random image URL: https://images.pexels.com/photos/18847930/pexels-photo-18847930.jpeg
Downloaded: composed_videos/24_20231102202232/220_random.jpeg
Random image URL: https://images.pexels.com/photos/18830088/pexels-photo-18830088.jpeg
Downloaded: composed_videos/24_20231102202232/221_random.jpeg
Random image URL: https://images.pexels.com/photos/18847930/pexels-photo-18847930.jpeg
Downloaded: composed_videos/24_20231102202232/222_random.jpeg
Current clip length: 10.71
Target length: 10.71
Clip is already the target duration.
Current clip length: 9.401
Target length: 9.401
Clip is already the target duration.
Current clip length: 9.484
Ta

In [73]:

processed_clips = project.video_clips_list
speech_clip = project.speech_clip

min_fps = 24

# Set the audio of each clip to None
for clip in processed_clips:
    clip.audio = None

# Concatenate the clips
final_clip_no_audio = concatenate_videoclips(processed_clips)
print(f"Final clip duration: {final_clip_no_audio.duration}")

final_clip_with_audio = final_clip_no_audio.set_audio(speech_clip)
final_clip_with_audio.write_videofile("final_vid_with_audio.mp4", audio_codec='aac', fps=min_fps)





Final clip duration: 44.565
Moviepy - Building video final_clip_with_audio.mp4.
MoviePy - Writing audio in final_clip_with_audioTEMP_MPY_wvf_snd.mp4


chunk:   0%|          | 0/983 [00:00<?, ?it/s, now=None]

MoviePy - Done.
Moviepy - Writing video final_clip_with_audio.mp4



Moviepy - Done !
Moviepy - video ready final_clip_with_audio.mp4


In [67]:

# final_clip = concatenate_videoclips(processed_clips)

# final_clip.write_videofile("output.mp4", audio_codec='aac', fps=min_fps)


# Iterate through each clip and find the minimum FPS
# for clip in processed_clips:
#     # print(f"Clip FPS: {clip.fps}")
#     if clip.fps < min_fps:
#         min_fps = clip.fps









# final_clip = final_clip.without_audio()

# speech_audio = project.get_project_data('speech_audio')
# downloads_folder = project.get_project_data('download_folder')

# speech_filepath = f"download_folder/speech.mp3"
# download_file_from_url(speech_audio, speech_filepath)

# speech_clip = AudioFileClip(speech_filepath)
# print(f"speech clip duration {speech_clip.duration}")


# final_clip = concatenate_videoclips(processed_clips)
# final_clip = final_clip.without_audio()
# print(f"final clip duration {final_clip.duration}")




# video_with_audio = final_clip.set_audio(speech_clip)


# output_filepath = f"{downloads_folder}/output_video.mp4"
# video_with_audio.write_videofile(output_filepath, audio_codec='aac', min_fps=min_fps)

Moviepy - Building video final_clip_with_audio.mp4.
MoviePy - Writing audio in final_clip_with_audioTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video final_clip_with_audio.mp4



Moviepy - Done !
Moviepy - video ready final_clip_with_audio.mp4


In [ ]:

def trim_clip(clip, start_trim, end_trim):
    """
    Trim a video clip to fit a new length between start_trim and end_trim.
    
    :param clip: VideoFileClip instance representing the video to trim.
    :param start_trim: Start trim point in seconds.
    :param end_trim: End trim point in seconds.
    
    :return: Adjusted video clip.
    """
    
    current_duration = clip.duration
    target_duration = end_trim - start_trim

    print(f"Current clip length: {current_duration}")
    print(f"Target length: {target_duration}")

    if current_duration == target_duration:
        print("Clip is already the target duration.")
        return clip
    elif current_duration > target_duration:
        print("Clip is longer than target duration. Trimming...")
        return clip.subclip(start_trim, end_trim)
    else:
        print("Clip is shorter than target duration. Slowing down to fit...")
        speed_factor = current_duration / target_duration
        return clip.speedx(speed_factor)

# Example usage
# clip = VideoFileClip("composed_videos/final_video123/0.mp4")
# new_clip = trim_clip(clip, start_trim=4, end_trim=8)
# new_clip.write_videofile("output_video.mp4")






def crop_clip(video_clip, width=900, height=900):
    """
    Crop a video to a specific size, maintaining its center.
    
    :param video_clip: VideoFileClip instance of the video.
    :param width: Desired width of the cropped video.
    :param height: Desired height of the cropped video.
    
    :return: Cropped (and possibly resized) VideoFileClip.
    """
    
    # If clip's width and height match the desired dimensions, return the clip
    if video_clip.w == width and video_clip.h == height:
        print("Video is already the desired size")
        return video_clip
    
    current_aspect_ratio = video_clip.w / video_clip.h
    target_aspect_ratio = width / height

    # Calculate the cropping dimensions based on the aspect ratios
    if current_aspect_ratio > target_aspect_ratio:
        crop_width = video_clip.h * target_aspect_ratio
        crop_height = video_clip.h
    else:
        crop_width = video_clip.w
        crop_height = video_clip.w / target_aspect_ratio

    # Crop the video clip
    cropped_clip = crop(video_clip, width=crop_width, height=crop_height, 
                        x_center=video_clip.w/2, y_center=video_clip.h/2)

    # Resize the cropped video to the desired dimensions
    resized_clip = cropped_clip.resize(width=width, height=height)

    return resized_clip


# Usage
# clip = VideoFileClip("composed_videos/final_video123/0.mp4")
# new_clip = crop_clip(clip, width=1600, height=900)
# new_clip.write_videofile("output_video_2.mp4")


In [ ]:



# # Adding a new scene
# new_scene = {'id': 101, 'name': 'New Scene', 'duration': 120}
# project.add_scene(new_scene)

# # Updating project data
# project.update_project_data('new_key', 'new_value')

# # Accessing specific project data
# print(project.get_project_data('new_key'))

# # Printing the updated project
# print(project)

# print(project.project_data)

In [ ]:
def get_file_extension(url):
    # Unquote the URL to handle any encoded characters
    url = unquote(url)
    # Parse the URL into its components
    parsed_url = urlparse(url)
    # Get the path component of the URL and then extract the file extension
    return os.path.splitext(parsed_url.path)[1][1:]


def download_file_from_url(url, save_path):
    """
    Download a file from the provided URL and save it to the specified path.

    Parameters:
    - url (str): The URL of the file to download.
    - save_path (str): The local path where the file should be saved.
    """
    
    # Ensure the directory structure exists
    directory = os.path.dirname(save_path)
    os.makedirs(directory, exist_ok=True)
    
    response = requests.get(url, stream=True)
    response.raise_for_status()

    with open(save_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)

    return save_path


# Downloads final media for all scenes in a project and saves the filepath to the project class

def download_all_files_for_project():
    filenames = []

    now = datetime.now()
    formatted_timestamp = now.strftime("%Y%m%d%H%M%S")

    project_id = project.project_id

    project_download_folder = f"composed_videos/{project_id}_{formatted_timestamp}"
    project.update_project_data('download_folder', project_download_folder)

    all_scenes = project.get_all_scenes()

    for scene in all_scenes:
        scene_id = scene['id']
        media_id = scene['final_clip']

        if media_id:
            media_info = funcs_supabase.select_data("media", "id", media_id)[0]
            url = media_info.get('url', None)
            media_type = media_info.get('type', None)
            url_remote = media_info.get('url_remote', None)

            # sometimes if url is missing but remote url is present
            if url is None and url_remote is not None:
                url = url_remote
        else:
            # If media_id is empty or None, use pexels.get_random_image()
            url = pexels_2.random_image()
            media_type = 'image'  # Assuming the media type as image

        url_extension = get_file_extension(url)
        if url_extension == '':
            if media_type == 'image':
                url_extension = 'png'
            elif media_type == 'video':
                url_extension = 'mp4'
            else:
                url_extension = 'png'

        filename = f"{project_download_folder}/{media_id}.{url_extension}" if media_id else f"{project_download_folder}/{scene_id}_random.{url_extension}"
        download_file_from_url(url, filename)

        # Save filename to class 
        project.update_scene(scene_id, 'downloaded_filename', filename)

        filenames.append(filename)

    return filenames






In [ ]:
def get_clips_for_project():
    processed_clips = []
    all_scenes = project.get_all_scenes()
    for scene in all_scenes:
        scene_id = scene['id']
        timings = scene['timings']
        length = timings['length']
        downloaded_filename = scene['downloaded_filename']


        if downloaded_filename.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
            clip = VideoFileClip(downloaded_filename)


        elif downloaded_filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            # Set the duration for how long you want each image to display
            clip = ImageClip(downloaded_filename, duration=length)

        else:
            print(f"Unsupported format: {downloaded_filename}")


        # Trim the clip to the desired length
        trimmed_clip = trim_clip(clip, start_trim=0, end_trim=length)

        height = project.get_project_data('height')
        width = project.get_project_data('width')


        if height is None:
            height = 1080

        if width is None:
            width = 1080

        resized_clip = crop_clip(trimmed_clip, width=900, height=900)

        processed_clips.append(resized_clip)

    return processed_clips



def make_final_vid(processed_clips):

    final_clip = concatenate_videoclips(processed_clips)
    min_fps = 60
    # Iterate through each clip and find the minimum FPS
    for clip in processed_clips:
        if clip.fps < min_fps:
            min_fps = clip.fps



    final_clip = final_clip.without_audio()

    speech_audio = project.get_project_data('speech_audio')
    downloads_folder = project.get_project_data('download_folder')

    speech_filepath = f"download_folder/speech.mp3"
    download_file_from_url(speech_audio, speech_filepath)

    speech_clip = AudioFileClip(speech_filepath)
    print(f"speech clip duration {speech_clip.duration}")


    final_clip = concatenate_videoclips(processed_clips)
    final_clip = final_clip.without_audio()
    print(f"final clip duration {final_clip.duration}")

 


    video_with_audio = final_clip.set_audio(speech_clip)


    output_filepath = f"{downloads_folder}/output_video.mp4"
    video_with_audio.write_videofile(output_filepath, audio_codec='aac', min_fps=min_fps)


In [ ]:
project_id = 28

import scene_timings
project = MovieProject(project_id)
project.load_data()

scenes = project.get_all_scenes()
for scene in scenes :
    if scene.get("timings") is None:
        scene_timings.process(project_id)

        project = MovieProject(project_id)
        project.load_data()




filenames = download_all_files_for_project()
processed_clips = get_clips_for_project()
make_final_vid(processed_clips)



In [ ]:
import file_management
url = "https://images.pexels.com/photos/20787/pexels-photo.sd?auto=compress&cs=tinysrgb&dpr=2&h=750&w=1260"
print(file_management.get_url_file_extension(url))